<h2 align="center">Creating Tables; Energy in Colombia from XM's API.</h2>  
By: Jordi Tarragó i Vidal 

The tables to be created directly in MariaDB with a structure equivalent to:  
USE EP_Colombia;  
CREATE TABLE PrecioBolsa (  
 Fecha DATETIME,  
 PrecBolsNaci DECIMAL(12,5),  
 PRIMARY KEY(Fecha)  
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;  
</br>
Daily values have DATE as datatype instead of DATETIME.  
</br>
Since XM updates at different time rates the different parameters, it was decided to make one table per parameter.  
</br>
After table creation, data from 2017 to 2022 will be loaded, data as of 2023 will be managed in a different way, with programmed [automated] updates. This will be done in a separate notebook for clarification purpuses

In [1]:
import pandas as pd
import sqlalchemy
import pymysql
from sqlalchemy import create_engine
from sqlalchemy import text

In [2]:
from dgxm import precio_horario
from dgxm import precio_diario

In [3]:
with open('db_string', 'r') as cadena:
    engine = create_engine(cadena.readline())

In [4]:
tables = pd.read_csv('Tablas.csv')

In [5]:
tables.head(10)

,Tabla,Freq,Fecha,Dato,Digitos,Decimales
0,PrecioBolsa,hourly,DATETIME,PrecBolsNaci,12,5
1,PrecioContReg,hourly,DATETIME,PrecPromContRegu,12,5
2,PrecioContNoReg,hourly,DATETIME,PrecPromContNoRegu,12,5
3,PrecioTX1,hourly,DATETIME,PrecBolsNaciTX1,12,5
4,PrecioEscasez,daily,DATE,PrecEsca,12,5
5,PrecioEscasezAct,daily,DATE,PrecEscaAct,12,5
6,VolumenUtilTotal,daily,DATE,VoluUtilDiarEner,18,5
7,VolumenUtilFrac,daily,DATE,PorcVoluUtilDiar,6,5
8,CapacidadUtilTotal,daily,DATE,CapaUtilDiarEner,18,5
9,DemandaSIN,daily,DATE,DemaSIN,18,5


In [6]:
crear = '''
CREATE TABLE {0} (
 Fecha {1},
 {2} DECIMAL({3},{4}),
 PRIMARY KEY(Fecha)
)ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
'''

In [7]:
with engine.connect() as conn:
    for i in range(len(tables)):
        sql = text('DROP TABLE IF EXISTS {}'.format(tables.iloc[i]['Tabla']))
        conn.execute(sql)
        sql2 = text(crear.format(tables.iloc[i]['Tabla'], tables.iloc[i]['Fecha'], tables.iloc[i]['Dato'], \
                                tables.iloc[i]['Digitos'], tables.iloc[i]['Decimales']))
        conn.execute(sql2)


In [8]:
#SHOW TABLES:
tables_mariadb = pd.read_sql(text('SHOW TABLES'), engine.connect())

In [9]:
tables_mariadb

,Tables_in_EP_Colombia
0,CapacidadUtilTotal
1,DemandaSIN
2,PrecioBolsa
3,PrecioContNoReg
4,PrecioContReg
5,PrecioEscasez
6,PrecioEscasezAct
7,PrecioTX1
8,VolumenUtilFrac
9,VolumenUtilTotal


In [10]:
#SHOW TABLES:
with engine.connect() as conn:
    for i in range(len(tables)):
        tables_temp = pd.read_sql(text('DESCRIBE {}'.format(tables.iloc[i]['Tabla'])), conn)
        print(tables.iloc[i]['Tabla']+': ')
        display(tables_temp)
    

PrecioBolsa: 


,Field,Type,Null,Key,Default,Extra
0,Fecha,datetime,NO,PRI,None,
1,PrecBolsNaci,"decimal(12,5)",YES,,None,


PrecioContReg: 


,Field,Type,Null,Key,Default,Extra
0,Fecha,datetime,NO,PRI,None,
1,PrecPromContRegu,"decimal(12,5)",YES,,None,


PrecioContNoReg: 


...

,Field,Type,Null,Key,Default,Extra
0,Fecha,date,NO,PRI,None,
1,PorcVoluUtilDiar,"decimal(6,5)",YES,,None,


CapacidadUtilTotal: 


,Field,Type,Null,Key,Default,Extra
0,Fecha,date,NO,PRI,None,
1,CapaUtilDiarEner,"decimal(18,5)",YES,,None,


DemandaSIN: 


,Field,Type,Null,Key,Default,Extra
0,Fecha,date,NO,PRI,None,
1,DemaSIN,"decimal(18,5)",YES,,None,


In [11]:
engine.dispose()